<a href="https://colab.research.google.com/github/jackychh7878/Colab_AI_Project/blob/main/Sentence_Embedding_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentence Embedding Project

-In this project, I'm going to demonstrate a sentence embedding process using BERT tokenizer and BERT model

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import random
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: set up CUDA

In [ ]:
random_seed = 42
random.seed(random_seed)
torch.manual_seed(random_seed)

if torch.cuda.is_available():
  torch.cuda.manual_seed_all(random_seed)

# Step 2: Get the tokenizer and BERT

In [ ]:
tokenizer=BertTokenizer.from_pretrained("bert-base-uncased")
model=BertModel.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

# Step 3: Encode -> Transformer -> Embedding

In [ ]:
text="HKUST provides the best AI Courses."
text2="Andrew Ng is a mentor of AI course"

# Encode the text
encoding=tokenizer.batch_encode_plus([text, text2], padding=True, truncation=True, return_tensors="pt", add_special_tokens=True)

In [ ]:
encoding

{'input_ids': tensor([[  101, 22563, 19966,  3640,  1996,  2190,  9932,  5352,  1012,   102],
        [  101,  4080, 12835,  2003,  1037, 10779,  1997,  9932,  2607,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
input_ids=encoding["input_ids"] #Here is encoding the entire sentence
attention_mask=encoding["attention_mask"]

with torch.no_grad():
  outputs=model(input_ids, attention_mask=attention_mask)
  word_embeddings=outputs.last_hidden_state

In [ ]:
word_embeddings.shape

torch.Size([2, 10, 768])

In [ ]:
word_embeddings

tensor([[[-0.2359,  0.0908,  0.1432,  ..., -0.3712,  0.6289,  0.2142],
         [ 0.4068,  0.1324,  0.1015,  ...,  0.3946,  1.0986,  0.7787],
         [ 0.0106, -0.3487, -0.0570,  ..., -0.0393,  0.6324,  0.3440],
         ...,
         [ 0.3638,  0.1477,  0.4736,  ...,  0.0716, -0.0542, -0.4902],
         [ 0.7459, -0.0018, -0.1368,  ...,  0.1813, -0.5323, -0.3482],
         [ 0.5212,  0.0960,  0.1371,  ...,  0.2454, -0.5806, -0.3377]],

        [[-0.2696,  0.1545,  0.0252,  ..., -0.2831,  0.2181,  0.2690],
         [-0.1839,  0.4657,  0.3063,  ...,  0.2246,  0.4411, -0.1533],
         [-0.7788, -0.1443,  0.8175,  ...,  0.1074, -0.8104,  0.3079],
         ...,
         [-0.6748,  0.1096,  0.2283,  ..., -0.6879, -0.2871, -0.3072],
         [-0.0585, -0.5729,  0.3418,  ...,  0.4114, -0.0389, -0.4060],
         [ 0.7268,  0.0924, -0.1320,  ...,  0.2224, -0.8283, -0.3562]]])

In [ ]:
input_ids[0][0]

tensor(101)

# Step 4: Decode the sentence

In [ ]:
decode=tokenizer.decode(input_ids[0], skip_special_tokens=True)

decode

'hkust provides the best ai courses.'

In [ ]:
decode=tokenizer.decode(input_ids[1], skip_special_tokens=True)

decode

'andrew ng is a mentor of ai course'